# Purpose:

```
   Create a databricks notebook (as base python)
   
   Investigate the Databricks DBFS 
   
   Create and read Parquet files, and understand them better 
   
   Create and read JSON files, display etc 
   
   ```

<br>

### Introduction:

#### Spark SQL provides support for both reading and writing Parquet files that automatically preserves the schema of the original data !

In [6]:
# Parquet is a columnar format that is supported by many other data processing systems. 

# Spark SQL provides support for both reading and writing Parquet files that automatically 
# preserves the schema of the original data. 

# When writing Parquet files, all columns are automatically converted to be nullable for compatibility reasons.

<br>

In [8]:
# lets confirm we have a spark context created
# if you know me, u know i always do these first couple steps
sc

Out[ 57 ]: <SparkContext master=local[8] appName=Databricks Shell>

In [9]:
# list all the methods for the spark context, in case you want to look around
dir(sc)   # then you just enter sc.<the method>

Out[ 58 ]: 
['PACKAGE_EXTENSIONS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_encryption_enabled',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_repr_html_',
 '_serialize_to_jvm',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addClusterWideLibraryToPath',
 'addFile',
 'addIsolatedLibraryPath',
 'addPyFile',
 'appName',
 'applicationId',
 'binaryFiles',
 'binaryRecords',
 'broadcast',
 'cancelAllJobs',
 'cancelJobGroup',
 'defaultMinPartitions',
 'defaultParallelism',
 'dump_profiles',
 'emptyRDD',
 'environment',
 'getConf',
 'getLocalProperty',
 'getOrCreate',
 'hadoopFile',
 'hadoopRDD',
 'init_batched_serializer',
 'master',
 'newAPIHadoopFile',
 'newAPIHadoopRDD',
 'parallelize',
 'pickleFile',
 'profiler_collector',
 'pythonExec',
 'pythonVer',
 'range',
 'runJob',
 'sequenceFile',
 'serializer',
 'setCheckpointDir',
 'setJobDescription',
 'setJobGroup',
 'setLocalProperty',
 'setLogLevel',
 'setSystemProperty',
 'show_profiles',
 'sparkHome',
 'sparkUser',
 'startTime',
 'statusTracker',
 'stop',
 'textFile',
 'uiWebUrl',
 'union',
 'version',
 'wholeTextFiles']

In [10]:
# what is my apache version ? 
sc.version

Out[ 60 ]: '2.4.0'

In [11]:
# what is my specific python version ? 
sc.pythonVer

Out[ 62 ]: '3.5'

In [12]:
# what is my spark context app name ? 
sc.appName

Out[ 64 ]: 'Databricks Shell'

In [13]:
# tell me my URL for seeing stages etc 
sc.uiWebUrl # etc etc type methods 

Out[ 65 ]: 'http://10.172.250.85:48228'

<br>

# Let's look at Parquet Files:

THE standard i think:
*  https://github.com/apache/parquet-format

In [17]:
# side note:  
# upload image to databricks DBFS
# it will show up under DBFS /FileStore/tables/<yourfilename>
# and then from there, IF you want to access the file for HTML image view:
#  /files/svm.jpg
# but for writing, it would be:
#  /FileStore/svm.jpg

Parquet is a columnar file format that provides optimizations to speed up queries and is a far more efficient file format than CSV or JSON.

Parquet stores binary data in a column-oriented way, where the values of each column are organized so that they are all adjacent, enabling better compression. It is especially good for queries which read particular columns from a “wide” (with many columns) table since only needed columns are read and IO is minimized.

displayHTML("<img src ='/files/tables/parquet.gif'>")

In [20]:
%scala 

// lets use scala to manually create a tiny array and save as an Apache Parquet file ! 

case class MyCaseClass(key: String, group: String, value: Int, someints: Seq[Int], somemap: Map[String, Int])

// paralellize an array of data, then toDF() method

val dataframe = sc.parallelize(Array(MyCaseClass("a", "vowels", 1, Array(1), Map("a" -> 1)),
  MyCaseClass("b", "consonants", 2, Array(2, 2), Map("b" -> 2)),
  MyCaseClass("c", "consonants", 3, Array(3, 3, 3), Map("c" -> 3)),
  MyCaseClass("d", "consonants", 4, Array(4, 4, 4, 4), Map("d" -> 4)),
  MyCaseClass("e", "vowels", 5, Array(5, 5, 5, 5, 5), Map("e" -> 5)))
).toDF()


// now write it to disk * AS parquet * 
// use:  dataframe method .write 
dataframe.write.mode("overwrite").parquet("/tmp/testParquet")  


// your output will include text like:
//  defined class MyCaseClass
//  dataframe: org.apache.spark.sql.DataFrame = [key: string, group: string ... 3 more fields]

defined class MyCaseClass
dataframe: org.apache.spark.sql.DataFrame = [key: string, group: string ... 3 more fields]

In [21]:
%scala
// i want to see my raw data please 
display(dataframe)

key,group,value,someints,somemap
a,vowels,1,List(1),Map(a -> 1)
b,consonants,2,"List(2, 2)",Map(b -> 2)
c,consonants,3,"List(3, 3, 3)",Map(c -> 3)
d,consonants,4,"List(4, 4, 4, 4)",Map(d -> 4)
e,vowels,5,"List(5, 5, 5, 5, 5)",Map(e -> 5)


In [22]:
%scala

// see how i use sqlContext to read the actual file ? 

val data = sqlContext.read.parquet("/tmp/testParquet")
// note:  use sqlContext.read.parquet to access 

// run this in databricks so you can see some of the cool outputs under the View Stage data

display(data)

key,group,value,someints,somemap
b,consonants,2,"List(2, 2)",Map(b -> 2)
c,consonants,3,"List(3, 3, 3)",Map(c -> 3)
d,consonants,4,"List(4, 4, 4, 4)",Map(d -> 4)
e,vowels,5,"List(5, 5, 5, 5, 5)",Map(e -> 5)
a,vowels,1,List(1),Map(a -> 1)


In [23]:
# You have spark context running, but you also have sqlContext as well...
sqlContext  # confirm it is running...

Out[ 71 ]: <pyspark.sql.context.HiveContext at 0x7faccb8a9e10>

In [24]:
# now lets use Python (not Scala), and read the parquet file you created

data2 = sqlContext.read.parquet("/tmp/testParquet")

display(data2)

# note in the real world, using non Databricks, you can just print out as well

key,group,value,someints,somemap
b,consonants,2,"List(2, 2)",Map(b -> 2)
c,consonants,3,"List(3, 3, 3)",Map(c -> 3)
d,consonants,4,"List(4, 4, 4, 4)",Map(d -> 4)
e,vowels,5,"List(5, 5, 5, 5, 5)",Map(e -> 5)
a,vowels,1,List(1),Map(a -> 1)


In [25]:
data2.sql_ctx

Out[ 77 ]: <pyspark.sql.context.HiveContext at 0x7faccb8a9e10>

In [26]:
# i want to know how many rows of data are in my data2 file:
data2.count()

Out[ 79 ]: 5

In [27]:
print(type(data2))  # this is a DataFrame fyi 

<class 'pyspark.sql.dataframe.DataFrame'>

In [28]:
# please list out my dataframe columns:
data2.columns

Out[ 98 ]: ['key', 'group', 'value', 'someints', 'somemap']

In [29]:
# i wish to output the column of group only:
display(data2.select('group'))

group
consonants
consonants
consonants
vowels
vowels


In [30]:
data2.schema

Out[ 107 ]: StructType(List(StructField(key,StringType,true),StructField(group,StringType,true),StructField(value,IntegerType,true),StructField(someints,ArrayType(IntegerType,true),true),StructField(somemap,MapType(StringType,IntegerType,true),true)))

##### Use SQL direct commands, which i really like:

*Create a temporary table and then we can query it*

In [33]:
%sql 
    CREATE TEMPORARY TABLE scalaTable
    USING parquet
    OPTIONS (
      path "/tmp/testParquet"
    )

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.analysis.TempTableAlreadyExistsException: Temporary view 'scalaTable' already exists;
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.createTempView(SessionCatalog.scala:520)
	at org.apache.spark.sql.execution.datasources.CreateTempViewUsing.run(ddl.scala:111)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:72)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:81)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:205)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:205)
	at org.apache.spark.sql.Dataset$$anonfun$55.apply(Dataset.scala:3424)
	at org.apache.spark.sql.Dataset$$anonfun$55.apply(Dataset.scala:3419)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:240)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:170)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$withAction(Dataset.scala:3419)
	at org.apache.spark.sql.Dataset. (Dataset.scala:205)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:89)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:696)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:716)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:88)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:34)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:34)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:141)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$8.apply(DriverLocal.scala:342)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$8.apply(DriverLocal.scala:319)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:47)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:271)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:47)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:319)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at scala.util.Try$.apply(Try.scala:192)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:639)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:485)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:597)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:390)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverW

In [34]:
%sql 
-- i love how you can use SQL directly in Databricks, its just freaking excellent
SELECT * FROM scalaTable

key,group,value,someints,somemap
b,consonants,2,"List(2, 2)",Map(b -> 2)
c,consonants,3,"List(3, 3, 3)",Map(c -> 3)
d,consonants,4,"List(4, 4, 4, 4)",Map(d -> 4)
e,vowels,5,"List(5, 5, 5, 5, 5)",Map(e -> 5)
a,vowels,1,List(1),Map(a -> 1)


In [35]:
%sql 
-- i want the data record row where the value is '2':
SELECT * FROM scalaTable WHERE value = 2

key,group,value,someints,somemap
b,consonants,2,"List(2, 2)",Map(b -> 2)


<br>
<br>

# Parquet Example #2

In [38]:
# lets look at a much larger file
# File uploaded to /FileStore/tables/userdata1.parquet

A good source of some parquet files:
* https://github.com/Teradata/kylo/tree/master/samples/sample-data/parquet

In [40]:
display(dbutils.fs.ls("dbfs:/FileStore/tables"))  # you can see your file there

path,name,size
dbfs:/FileStore/tables/Cbp/,Cbp/,0
dbfs:/FileStore/tables/README.md,README.md,2033
dbfs:/FileStore/tables/beer_data.csv,beer_data.csv,3143
dbfs:/FileStore/tables/boston_train.csv,boston_train.csv,24462
dbfs:/FileStore/tables/json.json,json.json,243
dbfs:/FileStore/tables/parquet.gif,parquet.gif,43589
dbfs:/FileStore/tables/pledge_of_allegiance.txt,pledge_of_allegiance.txt,174
dbfs:/FileStore/tables/userdata1.parquet,userdata1.parquet,113629


In [41]:
dataP = sqlContext.read.parquet("dbfs:/FileStore/tables/userdata1.parquet")

# -> Number of rows in the file: 1000
# -> Column details:
# column#		column_name		hive_datatype
# =====================================================
# 1		registration_dttm 	    timestamp
# 2		id 			            int
# 3		first_name 		        string
# 4		last_name 		        string
# 5		email 			        string
# 6		gender 			        string
# 7		ip_address 		        string
# 8		cc 			            string
# 9		country 		        string
# 10	birthdate 		        string
# 11	salary 			        double
# 12	title 			        string
# 13	comments 		        string



In [42]:
display(dataP)

registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
2016-02-03T07:55:29.000+0000,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
2016-02-03T17:04:03.000+0000,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2016-02-03T01:09:31.000+0000,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
2016-02-03T00:36:21.000+0000,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
2016-02-03T05:05:31.000+0000,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,null,,
2016-02-03T07:22:34.000+0000,6,Kathryn,White,kwhite5@google.com,Female,195.131.81.179,3583136326049310,Indonesia,2/25/1983,69227.11,Account Executive,
2016-02-03T08:33:08.000+0000,7,Samuel,Holmes,sholmes6@foxnews.com,Male,232.234.81.197,3582641366974690,Portugal,12/18/1987,14247.62,Senior Financial Analyst,
2016-02-03T06:47:06.000+0000,8,Harry,Howell,hhowell7@eepurl.com,Male,91.235.51.73,,Bosnia and Herzegovina,3/1/1962,186469.43,Web Developer IV,
2016-02-03T03:52:53.000+0000,9,Jose,Foster,jfoster8@yelp.com,Male,132.31.53.61,,South Korea,3/27/1992,231067.84,Software Test Engineer I,1E+02
2016-02-03T18:29:47.000+0000,10,Emily,Stewart,estewart9@opensource.org,Female,143.28.251.245,3574254110301671,Nigeria,1/28/1997,27234.28,Health Coach IV,


In [43]:
# I can poll the schema directly
dataP.schema

Out[ 153 ]: StructType(List(StructField(registration_dttm,TimestampType,true),StructField(id,IntegerType,true),StructField(first_name,StringType,true),StructField(last_name,StringType,true),StructField(email,StringType,true),StructField(gender,StringType,true),StructField(ip_address,StringType,true),StructField(cc,StringType,true),StructField(country,StringType,true),StructField(birthdate,StringType,true),StructField(salary,DoubleType,true),StructField(title,StringType,true),StructField(comments,StringType,true)))

In [44]:
# feeling lazy ? 

# upload parquet file, read it into dataframe in python, and then use 
# databrick's export 'Download csv' buttom (far right downward arrow) to
# download and view the csv form on your laptop...

<br>

# Let's examine JSON

In [47]:
# i manually uploaded this file called json.json
# sidenote: If your cluster is running Databricks Runtime 4.0 and above, you can read JSON 
# files in single-line or multi-line mode. In single-line mode, a file can be split into 
# many parts and read in parallel.

# lets confirm i can see it in my storage: 
display(dbutils.fs.ls("dbfs:/FileStore/tables"))

path,name,size
dbfs:/FileStore/tables/Cbp/,Cbp/,0
dbfs:/FileStore/tables/README.md,README.md,2033
dbfs:/FileStore/tables/beer_data.csv,beer_data.csv,3143
dbfs:/FileStore/tables/boston_train.csv,boston_train.csv,24462
dbfs:/FileStore/tables/json.json,json.json,243
dbfs:/FileStore/tables/pledge_of_allegiance.txt,pledge_of_allegiance.txt,174


In [48]:
# this already exists
spark

Out[ 87 ]: <pyspark.sql.session.SparkSession at 0x7faccb8a9908>

In [49]:
# this already exists
sqlContext

Out[ 88 ]: <pyspark.sql.context.HiveContext at 0x7faccb8a9e10>

In [50]:
# list methods, but without those annoying _ underlines 

for i in dir(sqlContext):
  if not i.startswith("_"): print(i)

cacheTable
clearCache
createDataFrame
createExternalTable
dropTempTable
getConf
getOrCreate
newSession
range
read
readStream
refreshTable
registerDataFrameAsTable
registerFunction
registerJavaFunction
setConf
sparkSession
sql
streams
table
tableNames
tables
udf
uncacheTable

In [51]:
randomDF = sqlContext.read.json("dbfs:/FileStore/tables/json.json")

In [52]:
# this is my json.json file i uploaded:
#  {"string":"string1","int":1,"array":[1,2,3],"dict": {"key": "value1"}}
#  {"string":"string2","int":2,"array":[2,4,6],"dict": {"key": "value2"}}
#  {"string":"string3","int":3,"array":[3,6,9],"dict": {"key": "value3", "extra_key":extra_value3"}}


# but the cool part is that Spark infers the schema automatically
randomDF.printSchema


Out[ 92 ]: <bound method DataFrame.printSchema of DataFrame[array: array<bigint>, dict: struct<extra_key:string,key:string>, int: bigint, string: string]>

In [53]:
display(randomDF)

array,dict,int,string
"List(1, 2, 3)","List(null, value1)",1,string1
"List(2, 4, 6)","List(null, value2)",2,string2
"List(3, 6, 9)","List(extra_value3, value3)",3,string3


<br>

#### Another JSON example:

In [56]:
import json
import requests

# You’ll need to make an API request to the JSONPlaceholder service, so just use the requests 
# package to do the heavy lifting.
response = requests.get("https://jsonplaceholder.typicode.com/todos")
todos = json.loads(response.text)


In [57]:
response.json()

Out[ 129 ]: 
[{'userId': 1, 'id': 1, 'completed': False, 'title': 'delectus aut autem'},
 {'userId': 1,
 'id': 2,
 'completed': False,
 'title': 'quis ut nam facilis et officia qui'},
 {'userId': 1, 'id': 3, 'completed': False, 'title': 'fugiat veniam minus'},
 {'userId': 1, 'id': 4, 'completed': True, 'title': 'et porro tempora'},
 {'userId': 1,
 'id': 5,
 'completed': False,
 'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum'},
 {'userId': 1,
 'id': 6,
 'completed': False,
 'title': 'qui ullam ratione quibusdam voluptatem quia omnis'},
 {'userId': 1,
 'id': 7,
 'completed': False,
 'title': 'illo expedita consequatur quia in'},
 {'userId': 1,
 'id': 8,
 'completed': True,
 'title': 'quo adipisci enim quam ut ab'},
 {'userId': 1,
 'id': 9,
 'completed': False,
 'title': 'molestiae perspiciatis ipsa'},
 {'userId': 1,
 'id': 10,
 'completed': True,
 'title': 'illo est ratione doloremque quia maiores aut'},
 {'userId': 1,
 'id': 11,
 'completed': True,
 'title': 'vero rerum temporibus dolor'},
 {'userId': 1,
 'id': 12,
 'completed': True,
 'title': 'ipsa repellendus fugit nisi'},
 {'userId': 1, 'id': 13, 'completed': False, 'title': 'et doloremque nulla'},
 {'userId': 1,
 'id': 14,
 'completed': True,
 'title': 'repellendus sunt dolores architecto voluptatum'},
 {'userId': 1,
 'id': 15,
 'completed': True,
 'title': 'ab voluptatum amet voluptas'},
 {'userId': 1,
 'id': 16,
 'completed': True,
 'title': 'accusamus eos facilis sint et aut voluptatem'},
 {'userId': 1,
 'id': 17,
 'completed': True,
 'title': 'quo laboriosam deleniti aut qui'},
 {'userId': 1,
 'id': 18,
 'completed': False,
 'title': 'dolorum est consequatur ea mollitia in culpa'},
 {'userId': 1,
 'id': 19,
 'completed': True,
 'title': 'molestiae ipsa aut voluptatibus pariatur dolor nihil'},
 {'userId': 1,
 'id': 20,
 'completed': True,
 'title': 'ullam nobis libero sapiente ad optio sint'},
 {'userId': 2,
 'id': 21,
 'completed': False,
 'title': 'suscipit repellat esse quibusdam voluptatem incidunt'},
 {'userId': 2,
 'id': 22,
 'completed': True,
 'title': 'distinctio vitae autem nihil ut molestias quo'},
 {'userId': 2,
 'id': 23,
 'completed': False,
 'title': 'et itaque necessitatibus maxime molestiae qui quas velit'},
 {'userId': 2,
 'id': 24,
 'completed': False,
 'title': 'adipisci non ad dicta qui amet quaerat doloribus ea'},
 {'userId': 2,
 'id': 25,
 'completed': True,
 'title': 'voluptas quo tenetur perspiciatis explicabo natus'},
 {'userId': 2, 'id': 26, 'completed': True, 'title': 'aliquam aut quasi'},
 {'userId': 2,
 'id': 27,
 'completed': True,
 'title': 'veritatis pariatur delectus'},
 {'userId': 2,
 'id': 28,
 'completed': False,
 'title': 'nesciunt totam sit blanditiis sit'},
 {'userId': 2, 'id': 29, 'completed': False, 'title': 'laborum aut in quam'},
 {'userId': 2,
 'id': 30,
 'completed': True,
 'title': 'nemo perspiciatis repellat ut dolor libero commodi blanditiis omnis'},
 {'userId': 2,
 'id': 31,
 'completed': False,
 'title': 'repudiandae totam in est sint facere fuga'},
 {'userId': 2,
 'id': 32,
 'completed': False,
 'title': 'earum doloribus ea doloremque quis'},
 {'userId': 2, 'id': 33, 'completed': False, 'title': 'sint sit aut vero'},
 {'userId': 2,
 'id': 34,
 'completed': False,
 'title': 'porro aut necessitatibus eaque distinctio'},
 {'userId': 2,
 'id': 35,
 'completed': True,
 'title': 'repellendus veritatis molestias dicta incidunt'},
 {'userId': 2,
 'id': 36,
 'completed': True,
 'title': 'excepturi deleniti adipisci voluptatem et neque optio illum ad'},
 {'userId': 2, 'id': 37, 'completed': False, 'title': 'sunt cum tempora'},
 {'userId': 2, 'id': 38, 'completed': False, 'title': 'totam quia non'},
 {'userId': 2,
 'id': 39,
 'completed': False,
 'title': 'doloremque quibusdam asperiores libero corrupti illum qui omnis'},
 {'userId': 2,
 'id': 40,
 'completed': True,
 'title': 'totam atque quo nesciunt'},
 {'userId': 3,
 'id': 41,
 'completed': False,
 'title': 'aliquid amet impedit consequatur aspernat

In [58]:
todos == response.json()

Out[ 130 ]: True

In [59]:
type(todos)

Out[ 131 ]: list

In [60]:
todos[:4]

Out[ 132 ]: 
[{'userId': 1, 'id': 1, 'completed': False, 'title': 'delectus aut autem'},
 {'userId': 1,
 'id': 2,
 'completed': False,
 'title': 'quis ut nam facilis et officia qui'},
 {'userId': 1, 'id': 3, 'completed': False, 'title': 'fugiat veniam minus'},
 {'userId': 1, 'id': 4, 'completed': True, 'title': 'et porro tempora'}]

In [61]:
display(todos)

completed,id,title,userId
false,1,delectus aut autem,1
false,2,quis ut nam facilis et officia qui,1
false,3,fugiat veniam minus,1
true,4,et porro tempora,1
false,5,laboriosam mollitia et enim quasi adipisci quia provident illum,1
false,6,qui ullam ratione quibusdam voluptatem quia omnis,1
false,7,illo expedita consequatur quia in,1
true,8,quo adipisci enim quam ut ab,1
false,9,molestiae perspiciatis ipsa,1
true,10,illo est ratione doloremque quia maiores aut,1


#### another JSON example

In [63]:
dbutils.fs.put("/tmp/test.json", """
{"string":"string1","int":1,"array":[1,2,3],"dict": {"key": "value1"}}
{"string":"string2","int":2,"array":[2,4,6],"dict": {"key": "value2"}}
{"string":"string3","int":3,"array":[3,6,9],"dict": {"key": "value3", "extra_key": "extra_value3"}}
""", True)

Wrote 243 bytes.
 Out[ 123 ]: True

In [64]:
testJsonData = sqlContext.read.json("/tmp/test.json")

display(testJsonData)

array,dict,int,string
"List(1, 2, 3)","List(null, value1)",1,string1
"List(2, 4, 6)","List(null, value2)",2,string2
"List(3, 6, 9)","List(extra_value3, value3)",3,string3


In [65]:
%sql 
    CREATE TEMPORARY TABLE jsonTable
    USING json
    OPTIONS (
      path "/tmp/test.json"
    )

In [66]:
%sql SELECT * FROM jsonTable


array,dict,int,string
"List(1, 2, 3)","List(null, value1)",1,string1
"List(2, 4, 6)","List(null, value2)",2,string2
"List(3, 6, 9)","List(extra_value3, value3)",3,string3


<br>

# Databricks Delta Examination

>  *lets look at a deeper examination of parquet and delta, and if we can make some improvements*

In [70]:
# note in the real world, using non Databricks, you can just print out as well

flights = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("/databricks-datasets/asa/airlines/2008.csv")

# see how the schema is inferred ???

In [71]:
# write a parquet-based table using this flights data 
# i specifically partition by ORIGIN
# once i do this, it will spread files around, and breakout by folders of Origin

flights.write.format("parquet").mode("overwrite").partitionBy("Origin").save("/tmp/flights_parquet")

In [72]:
# your output files
display(dbutils.fs.ls("dbfs:/tmp/flights_parquet"))  # you can see your file there

path,name,size
dbfs:/tmp/flights_parquet/Origin=ABE/,Origin=ABE/,0
dbfs:/tmp/flights_parquet/Origin=ABI/,Origin=ABI/,0
dbfs:/tmp/flights_parquet/Origin=ABQ/,Origin=ABQ/,0
dbfs:/tmp/flights_parquet/Origin=ABY/,Origin=ABY/,0
dbfs:/tmp/flights_parquet/Origin=ACK/,Origin=ACK/,0
dbfs:/tmp/flights_parquet/Origin=ACT/,Origin=ACT/,0
dbfs:/tmp/flights_parquet/Origin=ACV/,Origin=ACV/,0
dbfs:/tmp/flights_parquet/Origin=ACY/,Origin=ACY/,0
dbfs:/tmp/flights_parquet/Origin=ADK/,Origin=ADK/,0
dbfs:/tmp/flights_parquet/Origin=ADQ/,Origin=ADQ/,0


In [73]:
# 8 core files per origin, example shown:
display(dbutils.fs.ls("dbfs:/tmp/flights_parquet/Origin=DFW/"))  # you can see your file there

path,name,size
dbfs:/tmp/flights_parquet/Origin=DFW/_SUCCESS,_SUCCESS,0
dbfs:/tmp/flights_parquet/Origin=DFW/_committed_1227084317432497623,_committed_1227084317432497623,832
dbfs:/tmp/flights_parquet/Origin=DFW/_started_1227084317432497623,_started_1227084317432497623,0
dbfs:/tmp/flights_parquet/Origin=DFW/part-00000-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-421-79.c000.snappy.parquet,part-00000-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-421-79.c000.snappy.parquet,399190
dbfs:/tmp/flights_parquet/Origin=DFW/part-00001-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-422-79.c000.snappy.parquet,part-00001-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-422-79.c000.snappy.parquet,724010
dbfs:/tmp/flights_parquet/Origin=DFW/part-00002-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-423-80.c000.snappy.parquet,part-00002-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-423-80.c000.snappy.parquet,403831
dbfs:/tmp/flights_parquet/Origin=DFW/part-00003-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-424-80.c000.snappy.parquet,part-00003-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-424-80.c000.snappy.parquet,772614
dbfs:/tmp/flights_parquet/Origin=DFW/part-00004-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-425-79.c000.snappy.parquet,part-00004-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-425-79.c000.snappy.parquet,410121
dbfs:/tmp/flights_parquet/Origin=DFW/part-00005-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-426-80.c000.snappy.parquet,part-00005-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-426-80.c000.snappy.parquet,688253
dbfs:/tmp/flights_parquet/Origin=DFW/part-00006-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-427-79.c000.snappy.parquet,part-00006-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-427-79.c000.snappy.parquet,411121


In [74]:
# seeing display command in action, side note:
# from pyspark.sql.functions import avg
# diamonds_df = spark.read.csv("/databricks-datasets/Rdatasets/data-001/csv/ggplot2/diamonds.csv", header="true", inferSchema="true")
# display(diamonds_df.select("color","price").groupBy("color").agg(avg("price")))

In [75]:
# lets look at one single subFile

curiousDF = sqlContext.read.parquet("dbfs:/tmp/flights_parquet/Origin=DFW/part-00000-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-421-79.c000.snappy.parquet")

print("This individual dataset has the following number of rows: ", curiousDF.count())

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,2,9,6,926,933,1130,1154,OO,5927,N758SK,124,141,105,-24,-7,ORD,802,5,14,0,null,0,NA,NA,NA,NA,NA
2008,1,28,1,839,820,949,927,XE,2696,N12530,70,67,51,22,19,IAH,224,5,14,0,null,0,2,0,3,0,17
2008,2,9,6,1958,2007,2104,2129,OO,6210,N740SK,186,202,168,-25,-9,LAX,1235,6,12,0,null,0,NA,NA,NA,NA,NA
2008,1,24,4,1935,1930,2307,2258,XE,3069,N15980,152,148,129,9,5,CLE,1021,6,17,0,null,0,NA,NA,NA,NA,NA
2008,2,9,6,1228,1234,1348,1356,OO,6229,N740SK,200,202,166,-8,-6,LAX,1235,17,17,0,null,0,NA,NA,NA,NA,NA
2008,1,23,3,1301,1305,1420,1420,XE,2600,N14940,79,75,50,0,-4,IAH,224,17,12,0,null,0,NA,NA,NA,NA,NA
2008,2,10,7,1315,1315,1610,1630,OO,1980,N817SK,115,135,92,-20,0,ATL,732,9,14,0,null,0,NA,NA,NA,NA,NA
2008,1,26,6,1156,1200,1303,1310,XE,2905,N14938,67,70,45,-7,-4,IAH,224,7,15,0,null,0,NA,NA,NA,NA,NA
2008,2,10,7,1822,1759,2125,2115,OO,1998,N812SK,123,136,99,10,23,CVG,812,6,18,0,null,0,NA,NA,NA,NA,NA
2008,1,9,3,1920,1930,2245,2258,XE,3069,N15983,145,148,125,-13,-10,CLE,1021,3,17,0,null,0,NA,NA,NA,NA,NA


In [76]:
#

Once step 1 completes, the "flights" table contains details of US flights for a year.

Next in Step 2, we run a query that get top 20 cities with highest monthly total flights on first day of week.

In [78]:
# so when you open up the tmp folder, you see a few hundred files distributed by 
# origin id of like DFW airport.  

# eight files per original

# ex: /tmp/flights_parquet/Origin=DFW/part-00000-tid-1227084317432497623-9bdbeb42-5722-4f94-9755-6801a612974e-421-79.c000.snappy.parquet

#### step 2

In [80]:
# lets run a query:

from pyspark.sql.functions import count

# this has got to be a huge file, its consolidating everything 
flights_parquet = spark.read.format("parquet").load("/tmp/flights_parquet")

# query the huge file, breakout by day of week 1, groupby, and count totals
display(flights_parquet.filter("DayOfWeek=1").groupBy("Month","Origin").agg(count("*").alias("TotalFlights")).orderBy("TotalFlights", ascending=False).limit(20))

# this took 1.72 minutes ! 
# ran it again, and then it 2.46 minutes


Month,Origin,TotalFlights
6,ATL,6046
3,ATL,6019
12,ATL,5800
9,ATL,5722
6,ORD,5241
3,ORD,5072
9,ORD,4931
7,ATL,4894
8,ATL,4821
4,ATL,4798


In [81]:
print(type(flights_parquet))

<class 'pyspark.sql.dataframe.DataFrame'>

In [82]:
flights_parquet.count()
# thats over 7 million rows...

Out[ 172 ]: 7009728

In [83]:
# very very good command for seeing the schema you have in place for the DF
#############################a
flights_parquet.printSchema()
#############################

root
-- Year: integer (nullable = true)
-- Month: integer (nullable = true)
-- DayofMonth: integer (nullable = true)
-- DayOfWeek: integer (nullable = true)
-- DepTime: string (nullable = true)
-- CRSDepTime: integer (nullable = true)
-- ArrTime: string (nullable = true)
-- CRSArrTime: integer (nullable = true)
-- UniqueCarrier: string (nullable = true)
-- FlightNum: integer (nullable = true)
-- TailNum: string (nullable = true)
-- ActualElapsedTime: string (nullable = true)
-- CRSElapsedTime: string (nullable = true)
-- AirTime: string (nullable = true)
-- ArrDelay: string (nullable = true)
-- DepDelay: string (nullable = true)
-- Dest: string (nullable = true)
-- Distance: integer (nullable = true)
-- TaxiIn: string (nullable = true)
-- TaxiOut: string (nullable = true)
-- Cancelled: integer (nullable = true)
-- CancellationCode: string (nullable = true)
-- Diverted: integer (nullable = true)
-- CarrierDelay: string (nullable = true)
-- WeatherDelay: string (nullable = true)
-- NASDelay: string (nullable = true)
-- SecurityDelay: string (nullable = true)
-- LateAircraftDelay: string (nullable = true)
-- Origin: string (nullable = true)

#### step 3

Once step 2 completes, you can observe the latency with the standard "flights_parquet" table.

In step 3 and step 4, we do the same with a Databricks Delta table. This time, before running the query, we run the OPTIMIZE command with ZORDER to ensure data is optimized for faster retrieval.

In [86]:

# Step 3: Write a Databricks Delta based table using flights data
flights.write.format("delta").mode("overwrite").partitionBy("Origin").save("/tmp/flights_delta")


In [87]:

# Step 3 Continued: OPTIMIZE the Databricks Delta table

display(spark.sql("DROP TABLE  IF EXISTS flights"))

display(spark.sql("CREATE TABLE flights USING DELTA LOCATION '/tmp/flights_delta'"))
                  
display(spark.sql("OPTIMIZE flights ZORDER BY (DayofWeek)"))


path
null


In [88]:
# Step 4 : Rerun the query from Step 2 and observe the latency

flights_delta = spark.read.format("delta").load("/tmp/flights_delta")

display(flights_delta.filter("DayOfWeek=1").groupBy("Month","Origin").agg(count("*").alias("TotalFlights")).orderBy("TotalFlights", ascending=False).limit(20))

# its faster now 


Month,Origin,TotalFlights
6,ATL,6046
3,ATL,6019
12,ATL,5800
9,ATL,5722
6,ORD,5241
3,ORD,5072
9,ORD,4931
7,ATL,4894
8,ATL,4821
4,ATL,4798


In [89]:

#  HMMMMMMMM

#  it went from taking:  1.72 minutes (and then 2.46 mins)
#  to taking about    :  42.35 seconds
#  about 2.43X + faster...


<br>

## Appendix:  Access DBFS and looking around at functionality

<br>

In [93]:
display(dbutils.fs)

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

In [94]:

#  write a temp file to DBFS with python i/o apis
#  then print it out 
#  all python based (base notebook is python when created)

#write a file to DBFS using python i/o apis
with open("/dbfs/tmp/test_dbfs.txt", 'w') as f:
  f.write("Apache Spark is the bomb\n")
  f.write("Tom Bresee was the bomb\n")
  f.write("Now Databricks is the bomb\n")
  f.write("Goodbye.")

# read the file
with open("/dbfs/tmp/test_dbfs.txt", "r") as f_read:
  for line in f_read:
    print (line)
    
# df.write.text("/tmp/foo.txt")


Apache Spark is the bomb

Tom Bresee was the bomb

Now Databricks is the bomb

Goodbye.

In [95]:
# When you’re using Spark APIs, you reference files with "/mnt/training/file.csv" or 
# "dbfs:/mnt/training/file.csv". If you’re using local file APIs, you must provide 
# the path under /dbfs, for example: "/dbfs/mnt/training/file.csv". You cannot use 
# a path under dbfs when using Spark APIs.

In [96]:
%scala
// Now read the file you just created from scala

import scala.io.Source

val filename = "/dbfs/tmp/test_dbfs.txt"  // what you just created 

for (line <- Source.fromFile(filename).getLines()) {
  println(line)
}

Apache Spark is the bomb
Tom Bresee was the bomb
Now Databricks is the bomb
Goodbye.
import scala.io.Source
filename: String = /dbfs/tmp/test_dbfs.txt

In [97]:
# create a directory called foobar
dbutils.fs.mkdirs("/foobar/")

# now if you go to your main menu, and then click Import and Explore Data,
# you will see this new folder you just created ! 

Out[ 30 ]: True

In [98]:
# put the verbage into the file 
dbutils.fs.put("/foobar/baz.txt", "Hello, World!")

Wrote 13 bytes.
 Out[ 31 ]: True

In [99]:
# read the verbage you just created
dbutils.fs.head("/foobar/baz.txt")

Out[ 33 ]: 'Hello, World!'

In [100]:
# remove the file you just created
dbutils.fs.rm("/foobar/baz.txt")

Out[ 35 ]: False

In [101]:
# list out all the folders you have in DBFS ! 
# this is really a 'dir' level command ! 

display(dbutils.fs.ls("dbfs:/"))
# see how you can see the folder 'foobar' you created earlier ? 

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/cbp.vds/,cbp.vds/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/delta/,delta/,0
dbfs:/kdd/,kdd/,0
dbfs:/local_disk0/,local_disk0/,0
dbfs:/ml/,ml/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0


In [102]:
# lets look around
display(dbutils.fs.ls("dbfs:/tmp"))

path,name,size
dbfs:/tmp/hive/,hive/,0
dbfs:/tmp/test.json,test.json,243
dbfs:/tmp/testParquet/,testParquet/,0
dbfs:/tmp/test_dbfs.txt,test_dbfs.txt,84


In [103]:
# lets look around into the Parquet folder you created earlier ! ! ! 

display(dbutils.fs.ls("dbfs:/tmp/testParquet"))

path,name,size
dbfs:/tmp/testParquet/_SUCCESS,_SUCCESS,0
dbfs:/tmp/testParquet/_committed_193233552751229673,_committed_193233552751229673,606
dbfs:/tmp/testParquet/_started_193233552751229673,_started_193233552751229673,0
dbfs:/tmp/testParquet/part-00000-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-0-1-c000.snappy.parquet,part-00000-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-0-1-c000.snappy.parquet,807
dbfs:/tmp/testParquet/part-00001-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-1-1-c000.snappy.parquet,part-00001-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-1-1-c000.snappy.parquet,1606
dbfs:/tmp/testParquet/part-00003-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-3-1-c000.snappy.parquet,part-00003-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-3-1-c000.snappy.parquet,1667
dbfs:/tmp/testParquet/part-00004-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-4-1-c000.snappy.parquet,part-00004-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-4-1-c000.snappy.parquet,1667
dbfs:/tmp/testParquet/part-00006-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-6-1-c000.snappy.parquet,part-00006-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-6-1-c000.snappy.parquet,1667
dbfs:/tmp/testParquet/part-00007-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-7-1-c000.snappy.parquet,part-00007-tid-193233552751229673-96f8d3da-04a1-4c5e-a717-783a105255d0-7-1-c000.snappy.parquet,1631


In [104]:
# wanna know something odd ?   
# you CANT put comments into the same cell as a filesystem magic command ! 
# what the what, it will erorr out, so it has to be a stand alone cell ! 

In [105]:
%fs rm -r foobar

res11: Boolean = true

In [106]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/cbp.vds/,cbp.vds/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/delta/,delta/,0
dbfs:/kdd/,kdd/,0
dbfs:/local_disk0/,local_disk0/,0
dbfs:/ml/,ml/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0


In [107]:

try:
    import jira.client
    JIRA_IMPORTED = True

except ImportError:
    JIRA_IMPORTED = False